In [ ]:
#Use hashtags and simple regex to find training samples

In [ ]:
candidates = ['Trump','Clinton','Sanders','Cruz','Rubio']


In [ ]:
# positive example:
# Match (t:Tweet_2016_3_1)<-[:POSTS]-(u:UserSupportsSanders_2016_3_1)
# where exists(t.mentions_Sanders) and not exists(t.mentions_Clinton)
# and not exists(t.mentions_Rubio) and not exists(t.mentions_Trump)
# and not exists(t.mentions_Cruz) 
# return t limit 25

# negative example:
# Match (t:Tweet_2016_3_1)<-[:POSTS]-(u:UserSupportsTrump_2016_3_1)
# where exists(t.mentions_Sanders) and not exists(t.mentions_Clinton)
# and not exists(t.mentions_Rubio) and not exists(t.mentions_Trump)
# and not exists(t.mentions_Cruz)
# return t limit 25

pos_queries = {}
neg_queries = {}
opponents = {}

pos_queries['Sanders'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupportsSanders_{0})
where exists(t.mentions_Sanders) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Rubio) and not exists(t.mentions_Trump)
and not exists(t.mentions_Cruz) 
return t.text limit 100
"""

neg_queries['Sanders'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupports{1}_{0})
where exists(t.mentions_Sanders) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Rubio) and not exists(t.mentions_Trump)
and not exists(t.mentions_Cruz)
return t.text limit 25
"""

pos_queries['Clinton'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupportsClinton_{0})
where exists(t.mentions_Clinton) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Rubio) and not exists(t.mentions_Trump)
and not exists(t.mentions_Cruz) 
return t.text limit 100
"""

neg_queries['Clinton'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupports{1}_{0})
where exists(t.mentions_Clinton) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Rubio) and not exists(t.mentions_Trump)
and not exists(t.mentions_Cruz)
return t.text limit 25
"""

pos_queries['Trump'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupportsTrump_{0})
where exists(t.mentions_Trump) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Rubio) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Cruz) 
return t.text limit 100
"""

neg_queries['Trump'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupports{1}_{0})
where exists(t.mentions_Trump) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Rubio) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Cruz)
return t.text limit 25
"""

pos_queries['Rubio'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupportsRubio_{0})
where exists(t.mentions_Rubio) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Trump) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Cruz) 
return t.text limit 100
"""

neg_queries['Rubio'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupports{1}_{0})
where exists(t.mentions_Rubio) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Trump) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Cruz)
return t.text limit 25
"""

pos_queries['Cruz'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupportsCruz_{0})
where exists(t.mentions_Cruz) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Trump) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Rubio) 
return t.text limit 100
"""

neg_queries['Cruz'] = """Match (t:Tweet_{0})<-[:POSTS]-(u:UserSupports{1}_{0})
where exists(t.mentions_Cruz) and not exists(t.mentions_Sanders)
and not exists(t.mentions_Trump) and not exists(t.mentions_Clinton)
and not exists(t.mentions_Rubio)
return t.text limit 25
"""

opponents['Sanders'] = ['Clinton','Trump','Cruz','Rubio']
opponents['Clinton'] = ['Sanders','Trump','Cruz','Rubio']
opponents['Trump'] = ['Clinton','Sanders','Cruz','Rubio']
opponents['Rubio'] = ['Clinton','Sanders','Cruz','Trump']
opponents['Cruz'] = ['Clinton','Sanders','Trump','Rubio']


In [ ]:
partitions = ['2015_10_11', '2015_10_12', '2015_10_13', '2015_10_14', '2015_10_15',\
              '2016_1_26', '2016_1_27', '2016_1_28', '2016_1_29', '2016_1_30', '2016_2_3', \
              '2016_2_4', '2016_2_5', '2016_2_6', '2016_2_7', '2016_2_8', '2016_2_9', '2016_2_10',\
              '2016_2_11', '2016_2_12', '2016_2_13', '2016_2_14', '2016_2_15', \
              '2016_2_23', '2016_2_24', '2016_2_25', '2016_2_26', '2016_3_1', '2016_3_2',\
             '2016_3_3', '2016_3_4', '2016_3_5', '2016_3_6', '2016_3_7', '2016_3_8', '2016_3_9',\
             '2016_3_10', '2016_3_11']

# partitions = ['2016_2_24', '2016_2_25', '2016_2_26', '2016_3_1', '2016_3_2',\
#              '2016_3_3', '2016_3_4', '2016_3_5', '2016_3_6', '2016_3_7', '2016_3_8', '2016_3_9',\
#              '2016_3_10', '2016_3_11']



In [ ]:
from neo4j.v1 import GraphDatabase, basic_auth
import time
import csv

#driver = GraphDatabase.driver("bolt://sauce5.sdsc.edu", auth=basic_auth("neo4j", "lajolla"))
print 'opening driver...'
driver = GraphDatabase.driver("bolt://sauce5.sdsc.edu", auth=basic_auth("neo4j", "lajolla"))
#driver = GraphDatabase.driver("bolt://127.0.0.1")
print 'after opening driver...'

with open('sentimentsamples.csv', 'wb') as csvfile:
    tweetwriter = csv.writer(csvfile)
    
    for partition in partitions:
    #for partition in ['2016_3_2']:

        for candidate in candidates:

            start = time.time()

            pos_query = pos_queries[candidate].format(partition)
            #print pos_query

            session = driver.session()
            result = session.run(pos_query)

            for record in result:
                print partition, candidate, record[0]
                tweetwriter.writerow([partition,candidate,'pos','supporter',record[0].encode("utf-8")])

            #run negative cases
            for opponent in opponents[candidate]:

                neg_query = neg_queries[candidate].format(partition, opponent)
                #print neg_query

                result = session.run(neg_query)

                for record in result:
                    print partition, candidate, record[0]
                    tweetwriter.writerow([partition, candidate,'neg',(opponent+' supporter'),record[0].encode("utf-8")])

            session.close()

            end = time.time()
            print end - start

        